# Die Casting Automatic Defect Detection using CNN

## Project Description
🏭 Casting is a manufacturing process in which a liquid material is poured into a mould that contains a hollow cavity of the desired shape and then allowed to solidify. There are three types of casting processes: Sand Casting, Die Casting, and Lost Wax Casting.

🛠️ There are many types of defects in casting like holes, burr, shrinkage defects, mould material defects, pouring metal defects, metallurgical defects, etc.

🏭 A foundry that produces bearing bushes, in order of 10,000 parts per day has invested large funding to automate the process of finding defects in its casting production line. Currently, the inspection process is carried out manually by QC personnel. It is a very time-consuming process and due to human error, the process of rejecting defects is not very accurate. This can be the cause of rejection of an entire order which would lead to huge losses. The goal is to build a Machine Learning Model to eliminate this loss of revenue and make the QC process as accurate as possible.

📊 **Dataset**: All images are (512x512) pixels grey-scaled. There are two Folders, the Training, and the Test set, each containing two subfolders of “Defect” and “Okay” parts.  

📝 **Task**: To develop a deep-learning classification model for this problem. Evaluate and validate the model accuracy with this Test set. Later, I will  use the model to predict if a single unknown image (some examples in the Folder, New_set) is a defective or okay part. 

📂 **Source of the dataset**: (https://www.kaggle.com/ravirajsinh45/real-life-industrial-dataset-of-casting-product)


## Importing the Libraries

In [10]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

## Data Preprocessing

**Data Pre-processing:**

- 🧩 Split data into training and testing sets

- 🧩Preprocessing the Training Set:

    For the training set, I would apply a series of image transformations to enhance the model's ability to generalize. These transformations would include:

    - 🔄 Rescaling: The pixel values in the images are scaled down to a range between 0 and 1, which helps in standardizing the data.
    - 📏 Shear Range: Images would be subjected to shearing, which involves shifting one part of the image in a fixed direction, creating a sort of 'tilting' effect.
    - 🔍 Zoom Range: Random zooming would be applied to the images to simulate different perspectives.
    - ↔️ Horizontal Flip:  Some images would be horizontally flipped, which can help the model learn more robust features.

- 🧩Preprocessing the Test Set:

    - 🔄 Rescaling: For the test set, I would only perform rescaling to ensure consistency in the data. This is crucial to ensure that the model sees the data in a format similar to what it was trained on.
 

### Preprocessing the Training Set

In [11]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('Casting Dataset/Training_Set',
                                                 target_size = (64, 64),
                                                 batch_size = 32, # images are put in batches 32 images.
                                                 class_mode = 'binary') # Binary classification

Found 7383 images belonging to 2 classes.


In [12]:

len(training_set)

231

It is 231 because 7383/ 32 = ~ 231 . 

### Preprocessing the Test Set

In [13]:
test_datagen = ImageDataGenerator(rescale = 1./255) # we only do scaling for the testing set.
test_set = test_datagen.flow_from_directory('Casting Dataset/Test_Set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')# Binary classification

Found 863 images belonging to 2 classes.


## Building the CNN Model


**Building the CNN Model:**

1- Simply, I perform a series convolution + pooling operations, followed by flattening and a number of fully connected layers. Then, I compile and train the model.


2- In detail , a CNN model can be thought as a combination of two components: feature extraction part and the classification part. 
    - The convolution + pooling layers perform feature extraction. For example given an image, the convolution layer detects features such as two eyes, long ears, four legs, a short tail and so on. 
    - The fully connected layers then act as a classifier on top of these features, and assign a probability for the input image being a dog.

3- The convolution + pooling layers as powerhouse:
- The convolution layers are the main powerhouse of a CNN model. Automatically detecting meaningful features given only an image and a label is not an easy task.
- The convolution layers learn such complex features by building on top of each other. 
- The first layers detect edges, the next layers combine them to detect shapes, to following layers merge this information to infer that this is a nose. To be clear, the CNN doesn’t know what a nose is. By seeing a lot of them in images, it learns to detect that as a feature. The fully connected layers learn how to use these features produced by convolutions in order to correctly classify the images.


### Step 1 - Initialising the Model

In [14]:
Model = tf.keras.models.Sequential()

### Step 2 - Adding First Convolution Layer 

In [15]:
Model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

### Step 3 - Pooling the First Layer

I use a 2x2 pooling window (i.e. pool_size=2) as it’s the most common.

In [16]:
Model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=1))

### Step 4 - Adding a Second Convolutional Layer

In [17]:
Model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))

### Step 5 - Pooling the Second Layer

In [18]:
Model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=1))

### Step 6 - Flattening

Remember that the output of both convolution and pooling layers are 3D volumes, but a fully connected layer expects a 1D vector of numbers. So I flatten the output of the final pooling layer to a vector and that becomes the input to the fully connected layer. Flattening is simply arranging the 3D volume of numbers into a 1D vector

In [19]:
Model.add(tf.keras.layers.Flatten())

### Step 7 - Full Connection

In [20]:
Model.add(tf.keras.layers.Dense(units=256, activation='relu'))

### Step 8 - Output Layer

In [21]:
Model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

### Step 9 - Compiling the CNN

In [22]:
Model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

## Training the CNN and Evaluation

In [14]:
Model.fit(x = training_set, validation_data = test_set, epochs = 50)

Epoch 1/50
231/231 [==============================] - 72s 310ms/step - loss: 0.7897 - accuracy: 0.6060 - val_loss: 0.6471 - val_accuracy: 0.6060
Epoch 2/50
231/231 [==============================] - 47s 203ms/step - loss: 0.5906 - accuracy: 0.6718 - val_loss: 0.5144 - val_accuracy: 0.7323
Epoch 3/50
231/231 [==============================] - 47s 201ms/step - loss: 0.4246 - accuracy: 0.8025 - val_loss: 0.5865 - val_accuracy: 0.6964
Epoch 4/50
231/231 [==============================] - 46s 200ms/step - loss: 0.3537 - accuracy: 0.8465 - val_loss: 0.3072 - val_accuracy: 0.8505
Epoch 5/50
231/231 [==============================] - 46s 200ms/step - loss: 0.2791 - accuracy: 0.8807 - val_loss: 0.9737 - val_accuracy: 0.6570
Epoch 6/50
231/231 [==============================] - 46s 199ms/step - loss: 0.2441 - accuracy: 0.8994 - val_loss: 0.2213 - val_accuracy: 0.9050
Epoch 7/50
231/231 [==============================] - 46s 200ms/step - loss: 0.2095 - accuracy: 0.9141 - val_loss: 0.3698 - val_ac

Note: val_accuracy is really the accracy of the confusion matrix.

## Making a Prediction

In [16]:
training_set.class_indices

{'Defect': 0, 'Ok': 1}

In [15]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('Casting Dataset/InLineProduction_Detection/Part10.jpeg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = Model.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
  prediction = 'Part is Okay.'
else:
  prediction = 'Part is Defective!'
print(prediction)

Part is Defective!
